# Custom Cnns

A few table custom CNNs were created and are available in the models module of the utils package.

- "baseline_cnn"
- "deeper_cnn"
- "simple_cnn"
- "GuaxinimCNN"


## Environment Setup

In [ ]:
import os
import wandb
from dotenv import load_dotenv

# load variables from env
load_dotenv()

# Start weights and biases
wandb.login()


In [ ]:
import tensorflow as tf

# Enable memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ Enabled memory growth for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)

# Prevent Out Of Memory
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

## Reading the Dataset

In [ ]:
# Read the dataset
from utils.data import load_data

X_train, y_train, X_test, y_test, labels = load_data()
n_labels = len(labels)

## Raccoon CNN

This Cnn was engineered to use small images. To achieve that I tested with removing Pooling layers and replacing them with dropout.

![](images/tiredRaccoonCNN.png)

### Config and Run Experiment

In [ ]:
from utils.config import WandbConfig, ExperimentConfig

config = ExperimentConfig(input_shape=(32,32,3),
                          epochs=20,
                          batch_size=32,
                          model_name="GuaxinimCNN",
                          normalize=False)

experiment_name = f"{config.model_name}_{config.input_shape[0]}_{config.batch_size}"

wandb_config = WandbConfig(project_name="deep-learning-cifar10-classification_custom",
                           experiment_name=experiment_name)


In [ ]:
from utils.train import run_experiment

(train_ds,
 val_ds,
 test_ds,
 model,
 history,
 run) = run_experiment(X_train, y_train,
               X_test, y_test,
               wandb_config=wandb_config,
               config=config,
               class_names = labels)


### Eval

In [ ]:
import numpy as np
from utils.eval import get_metrics_report, plot_confusion_matrix

# Predictions
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
report = get_metrics_report(y_test, y_pred, labels)
plot_confusion_matrix(y_test, y_pred, labels)

In [ ]:
from utils.eval import plot_predictions, plot_wrong_predictions

plot_predictions(model, test_ds, labels)

plot_wrong_predictions(model, test_ds, labels)

## DeeperCNN

Was designed to compare results with the other CNNs using the standard practices of CNNs design.
To run this model use the same structure for the GuaxinimCNN (Raccoon CNN), only changing the model name. 


![](images/deeperCNN_.png)

## 